In [38]:
import json
import pandas as pd
import gzip

from collections import Counter

In [41]:
events = []
# File from: wget http://data.gharchive.org/2018-07-01-15.json.gz
with gzip.open('MetaGithub/2018-07-01-15.json.gz') as f:
    for line in f:
        events.append(json.loads(line.strip()))

In [44]:
Counter((event['type'] for event in events));

In [45]:
issues = [event for event in events if event['type'] == 'IssuesEvent']
comments = [event for event in events if event['type'] == 'IssueCommentEvent']
CreateEvents = [event for event in events if event['type'] == 'CreateEvent']

In [52]:
actors=[]
#Hold a list of all the usernames and repo names
#This is needed in order to pull README.md files
names=[]
## pull all of the createEvents in order to get UserID and Repo name for 
# calls to get Readme Content
CreateEvents = [event for event in events if event['type'] == 'CreateEvent']
for event in range(1,len(CreateEvents)):
    thisEvent= CreateEvents[event]
    actors.append(thisEvent['repo'])
    for name in range(0,len(actors)):
        thisName=actors[name]
        names.append(thisName['name'])

In [58]:
CreateEvents[1]

{'id': '7903170476',
 'type': 'CreateEvent',
 'actor': {'id': 40721477,
  'login': 'okaysage',
  'display_login': 'okaysage',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/okaysage',
  'avatar_url': 'https://avatars.githubusercontent.com/u/40721477?'},
 'repo': {'id': 139338168,
  'name': 'okaysage/vector',
  'url': 'https://api.github.com/repos/okaysage/vector'},
 'payload': {'ref': None,
  'ref_type': 'repository',
  'master_branch': 'master',
  'description': None,
  'pusher_type': 'user'},
 'public': True,
 'created_at': '2018-07-01T15:00:00Z'}

In [61]:
## I still need to use regex to parse the usernames and repo names
testNames=names[:10]
testNames[2]

'Rsaliu/Quiz-App'

In [62]:
from github import Github
# First create a Github instance:
# I will change this to token access soon; right now you can see how the 
# code works by entering : g = Github("username", "password")
g = Github("username", "password")

## This code will pull the README.md files for all of the repos and usernames:
# I need to write exceptions for when the repo is empty, as well as when the 
#repo does not have a README.md file (some repos have just README with no
#extension)
# For an example of each of these scenarios use testNames[1] to see error outpur
# and use testNames[2] to see the contents of a usable README.md
thisRepo=g.get_repo(testNames[2])
thisRepo
readMe=thisRepo.get_contents('README.md')
#print(readMe)
type(readMe)
readMe.decoded_content

b'## Project Title\n###### Nigeria Quiz App (Google Africa Challenge Scholarship)\n\n## Prerequisites\n###### The software can run on minimum of android API level 9 (Gingerbread) and is targeted at API level 24 (Marshhmallow)\n\n## Installing\n\n###### The APK file can be downloaded at https://drive.google.com/open?id=11aftgu7d-FYVpIzPCj27ClLc09aKrzuY\n\n## Built With\n\n###### Android Studio 2.1.2\n\n## Versioning\n\n###### This is version 1.0.0 of this software\n\n## Authors\n###### Saliu Rilwan Adebayo\n\n## Acknowledgments\n###### Hat tip to anyone whose code was used\n###### Google Africa Challenge Scholarship\n###### Udacity\n###### etc\n\n'